# NOTE: This notebook is taken from the GitHub repo of the LA Times datadesk:

Original Title: california-coronavirus-data examples

By [Ben Welsh](https://palewi.re/who-is-ben-welsh)

A demonstration of how to use Python to work with the Los Angeles Times' independent tally of coronavirus cases in California published on GitHub at [datadesk/california-coronavirus-data](https://github.com/datadesk/california-coronavirus-data#state-cdph-totalscsv). To run this notebook immediately in the cloud,  click the [Binder](https://mybinder.org/) launcher below.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/datadesk/california-coronavirus-data/master?urlpath=lab/tree/notebooks/examples.ipynb)

## Subnotes: 

* Notebook and data were retrieved on April 23, 2021
* Modifications for the IDRE workshop are made at the very bottom below the header "IDRE Workshop Additions"
* The Binder link for interacting with the IDRE workshop materials is [Here](https://mybinder.org/v2/gh/benjum/idre-spring21-python-data-viz-2/HEAD)

In [ ]:
# commented out for IDRE class
# %load_ext lab_black

## Import Python tools

Our data analysis and plotting tools

In [ ]:
import pandas as pd
import altair as alt

Customizations to the Altair theme

In [ ]:
import altair_latimes as lat

In [ ]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")

In [ ]:
alt.data_transformers.disable_max_rows()

## Import data

Read in the agency totals

In [ ]:
agency_df = pd.read_csv("data/latimes-agency-totals.csv", parse_dates=["date"])

In [ ]:
agency_df.head()

In [ ]:
agency_df.info()

## Aggregate data

### By state

Lump all the agencies together and you get the statewide totals.

In [ ]:
state_df = (
    agency_df.groupby(["date"])
    .agg({"confirmed_cases": "sum", "deaths": "sum"})
    .reset_index()
)

In [ ]:
state_df.head()

In [ ]:
state_df.info()

### By county

Three cities &mdash; Berkeley, Long Beach and Pasadena &mdash; run independent public health departments. Calculating county-level totals requires grouping them with their local peers.

In [ ]:
county_df = (
    agency_df.groupby(["date", "county"])
    .agg({"confirmed_cases": "sum", "deaths": "sum"})
    .reset_index()
)

In [ ]:
county_df.head()

In [ ]:
county_df.info()

## Chart the statewide totals over time

In [ ]:
# Create a base chart with the common x-axis
chart = alt.Chart(state_df).encode(x=alt.X("date:T", title=None))

# Create the cases line
cases = chart.mark_line(color=lat.palette["default"]).encode(
    y=alt.Y("confirmed_cases:Q", title="Confirmed cases")
)

# Create the deaths line
deaths = chart.mark_line(color=lat.palette["schemes"]["ice-7"][3]).encode(
    y=alt.Y("deaths:Q", title="Deaths")
)

# Combine them into a single chart
(cases & deaths).properties(title="Statewide cumulative totals")

## Chart the county totals

First on a linear scale

In [ ]:
# Create the base chart
chart = (
    alt.Chart(county_df)
    .mark_line()
    .encode(
        x=alt.X("date:T", title=None),
        color=alt.Color("county:N", title="County", legend=None),
    )
)

# The cases line
cases = chart.encode(y=alt.Y("confirmed_cases:Q", title="Confirmed cases"),)

# The deaths line
deaths = chart.mark_line().encode(y=alt.Y("deaths:Q", title="Deaths"),)

# Combined into a chart
(cases & deaths).properties(title="Cumulative totals by county")

Again on a logarithmic scale

In [ ]:
# Make a base chart
chart = (
    alt.Chart(county_df)
    .mark_line()
    .encode(
        x=alt.X("date:T", title=None),
        color=alt.Color("county:N", title="County", legend=None),
    )
)

# The cases lines
cases = chart.transform_filter(alt.datum.confirmed_cases > 0).encode(
    y=alt.Y("confirmed_cases:Q", scale=alt.Scale(type="log"), title="Confirmed cases"),
)

# The deaths lines
deaths = chart.transform_filter(alt.datum.deaths > 0).encode(
    y=alt.Y("deaths:Q", scale=alt.Scale(type="log"), title="Deaths"),
)

# Slapping them together
(cases & deaths).properties(title="Cumulative totals by county")

A common technique for clarifying these charts to begin each line on the day the county hit a minimum number. Let's try it with 10.

In [ ]:
day_10_df = (
    county_df[
        # Filter down to only days with 10 or more cumulative cases
        county_df.confirmed_cases
        >= 10
    ]
    .groupby(
        # And then get the minimum date for each county
        "county"
    )
    .date.min()
    .reset_index()
)

Merge that date to each row in the data.

In [ ]:
county_date_diff_df = county_df.merge(
    day_10_df, how="inner", on="county", suffixes=["", "_gte_10_cases"]
)

Calculate each day's distance from its tenth day.

In [ ]:
county_date_diff_df["days_since_10"] = (
    county_date_diff_df.date - county_date_diff_df.date_gte_10_cases
).dt.days

Chart it.

In [ ]:
alt.Chart(county_date_diff_df).transform_filter(
    # Only keep everything once they hit 10 cases
    alt.datum.days_since_10
    >= 0
).mark_line().encode(
    x=alt.X("days_since_10:O", title="Days since 10th case"),
    y=alt.Y("confirmed_cases:Q", scale=alt.Scale(type="log"), title="Confirmed cases"),
    color=alt.Color("county:N", title="County", legend=None),
).properties(
    title="Cumulative totals by county"
)

## County trends on a linear 'Pez' plot

Fill in any date gaps so that every county has a row for every date.

In [ ]:
backfilled_county_df = (
    county_df.set_index(["county", "date"])
    .unstack("county")
    .fillna(0)
    .stack("county")
    .reset_index()
)

Calculate the rolling change in each county.

In [ ]:
chronological_county_df = backfilled_county_df.sort_values(["county", "date"])

Calculate the daily change in each county.

In [ ]:
chronological_county_df["new_confirmed_cases"] = chronological_county_df.groupby(
    "county"
).confirmed_cases.diff()

Let's chill that out as a seven-day average.

In [ ]:
chronological_county_df["new_confirmed_cases_rolling_average"] = (
    chronological_county_df.groupby("county")
    .new_confirmed_cases.rolling(7)
    .mean()
    .droplevel(0)
)

Make the chart.

In [ ]:
alt.Chart(chronological_county_df, title="New cases by day").mark_rect(
    stroke=None
).encode(
    x=alt.X(
        "date:O", axis=alt.Axis(ticks=False, grid=False, labels=False,), title=None
    ),
    y=alt.Y(
        "county:N",
        title="County",
        axis=alt.Axis(ticks=False, grid=False, labelPadding=5),
    ),
    color=alt.Color(
        "new_confirmed_cases_rolling_average:Q",
        scale=alt.Scale(
            type="threshold", domain=[0, 3, 10, 25, 50, 100, 500], scheme="blues"
        ),
        title="New cases (7-day average)",
    ),
).properties(
    height=800
)

## Chart new cases and deaths

Calculate the number of new cases each day using panda's [diff](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.diff.html) method.

In [ ]:
state_df["new_confirmed_cases"] = state_df.confirmed_cases.diff()

Do the same for deaths

In [ ]:
state_df["new_deaths"] = state_df.deaths.diff()

Now calculate the moving seven-day average of each using panda's [rolling](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) method.

In [ ]:
state_df["new_confirmed_cases_rolling_average"] = state_df.new_confirmed_cases.rolling(
    7
).mean()

In [ ]:
state_df["new_deaths_rolling_average"] = state_df.new_deaths.rolling(7).mean()

Put it all together on the chart 

In [ ]:
# One base chart object with the data they all share
chart = alt.Chart(state_df).encode(x=alt.X("date:T", title=None),)

# The new cases bars
cases_bars = chart.mark_bar(color=lat.palette["default"]).encode(
    y=alt.Y("new_confirmed_cases:Q", title="New confirmed cases")
)

# The cases rolling average
cases_line = chart.mark_line(color=lat.palette["accent"]).encode(
    y=alt.Y("new_confirmed_cases_rolling_average:Q", title="7-day average")
)

# The new deaths bars
deaths_bars = chart.mark_bar(color=lat.palette["schemes"]["ice-7"][3]).encode(
    y=alt.Y("new_deaths:Q", title="New deaths")
)

# The deaths rolling average
deaths_line = chart.mark_line(color=lat.palette["schemes"]["ice-7"][6]).encode(
    y=alt.Y("new_deaths_rolling_average:Q", title="7-day average")
)

# Combine it all together into one paired chart
((cases_bars + cases_line) & (deaths_bars + deaths_line)).properties(
    title="New case and deaths statewide by day"
)

Now do it by county

In [ ]:
chronological_county_df.head()

Try it by county

In [ ]:
alt.Chart(chronological_county_df, title="New cases by day").mark_line().encode(
    x=alt.X("date:O", axis=alt.Axis(ticks=False, grid=False, labels=False), title=None),
    y=alt.Y("new_confirmed_cases_rolling_average:Q", title="7-day average"),
    color=alt.Color("county:N", title="County", legend=None),
)

Create a statistic to measure recent changes in new cases

In [ ]:
chronological_county_df.tail(14)

In [ ]:
chronological_county_df[
    "new_confirmed_cases_rolling_average_two_week_pct_change"
] = chronological_county_df.groupby(
    "county"
).new_confirmed_cases_rolling_average.pct_change(
    14
)

In [ ]:
latest_county_df = chronological_county_df[
    chronological_county_df.date == chronological_county_df.date.max()
]

In [ ]:
biggest_county_jumps = latest_county_df[
    latest_county_df.new_confirmed_cases_rolling_average >= 25
].sort_values(
    "new_confirmed_cases_rolling_average_two_week_pct_change", ascending=False
)

In [ ]:
def facet_wrap(subplts, plots_per_row):
    rows = [
        subplts[i : i + plots_per_row] for i in range(0, len(subplts), plots_per_row)
    ]
    compound_chart = alt.hconcat()
    for r in rows:
        rowplot = alt.vconcat()  # start a new row
        for item in r:
            rowplot |= item  # add suplot to current row as a new column
        compound_chart &= rowplot  # add the entire row of plots as a new row
    return compound_chart

In [ ]:
chart_list = []
for county in list(biggest_county_jumps.head(12).county):
    this_df = chronological_county_df[chronological_county_df.county == county]
    chart = alt.Chart(this_df, title=county).encode(
        x=alt.X("date:T", title=None, axis=None),
    )
    lines = chart.mark_line(color=lat.palette["accent"]).encode(
        y=alt.Y("new_confirmed_cases_rolling_average:Q", title=None,),
    )
    bars = chart.mark_bar(color=lat.palette["default"], opacity=0.33).encode(
        y=alt.Y("new_confirmed_cases:Q", title="New confirmed cases",),
    )
    chart_list.append((bars + lines).properties(height=200, width=250))
facet_wrap(chart_list, plots_per_row=4)

In [ ]:
chart_list = []
for county in list(biggest_county_jumps.tail(12).county):
    this_df = chronological_county_df[chronological_county_df.county == county]
    chart = alt.Chart(this_df, title=county).encode(
        x=alt.X("date:T", title=None, axis=None),
    )
    lines = chart.mark_line(color=lat.palette["accent"]).encode(
        y=alt.Y("new_confirmed_cases_rolling_average:Q", title=None,),
    )
    bars = chart.mark_bar(color=lat.palette["default"], opacity=0.33).encode(
        y=alt.Y("new_confirmed_cases:Q", title="New confirmed cases",),
    )
    chart_list.append((bars + lines).properties(height=200, width=250))
facet_wrap(chart_list, plots_per_row=4)

In [ ]:
biggest_county_jumps.new_confirmed_cases_rolling_average_two_week_pct_change.describe()

In [ ]:
biggest_county_jumps[
    biggest_county_jumps.new_confirmed_cases_rolling_average_two_week_pct_change < 0
]

# IDRE Workshop Additions

In [ ]:
alt.Chart(chronological_county_df, title="New cases by day").mark_line().encode(
    x=alt.X("date:O", axis=alt.Axis(ticks=False, grid=False, labels=False), title=None),
    y=alt.Y("new_confirmed_cases_rolling_average:Q", title="7-day average"),
    color=alt.Color("county:N", title="County", legend=None),
)

In [ ]:
selopac = alt.selection_single(fields=['county'],bind='legend')
how_to_select = 'CLICK ON COUNTY IN LEGEND TO SELECT'
chronological_county_df_minusLA = chronological_county_df[chronological_county_df['county']!='Los Angeles']

alt.Chart(chronological_county_df_minusLA, title="New cases by day"+how_to_select).mark_line().encode(
    x=alt.X("date:O", axis=alt.Axis(ticks=False, grid=False, labels=False), title=None),
    y=alt.Y("new_confirmed_cases_rolling_average:Q", title="7-day average"),
    color=alt.Color("county:N", title="County"),
    opacity=alt.condition(selopac, alt.value(1), alt.value(0.1))
).add_selection(selopac)

In [ ]:
# top 20 counties in terms of mean new confirmed cases
top20=list(chronological_county_df.groupby('county')['new_confirmed_cases'].mean().sort_values(ascending=False)[:20].index)

In [ ]:
selopac = alt.selection_single(on='mouseover',fields=['county'],bind='legend') 
how_to_select = 'MOVE MOUSE OVER LINE TO SELECT LINE'
chronological_county_df_top20 = chronological_county_df[chronological_county_df['county'].isin(top20)]

alt.Chart(chronological_county_df_top20, title="New cases by day"+how_to_select).mark_line().encode(
    x=alt.X("date:O", axis=alt.Axis(ticks=False, grid=False, labels=False), title=None),
    y=alt.Y("new_confirmed_cases_rolling_average:Q", title="7-day average"),
    color=alt.Color("county:N", title="County"),
    opacity=alt.condition(selopac, alt.value(1), alt.value(0.1))
).add_selection(selopac)

In [ ]:
selopac = alt.selection_single(fields=['county'],bind='legend') 
how_to_select = 'CLICK ON COUNTY IN LEGEND TO SELECT'
chronological_county_df_top20 = chronological_county_df[chronological_county_df['county'].isin(top20)]
chronological_county_df_top20 = chronological_county_df_top20[chronological_county_df_top20.date > '2021-03-01']

alt.Chart(chronological_county_df_top20, title="New cases by day"+how_to_select).mark_line().encode(
    x=alt.X("date:O", axis=alt.Axis(ticks=False, grid=False, labels=False), title=None),
    y=alt.Y("new_confirmed_cases_rolling_average:Q", title="7-day average"),
    color=alt.Color("county:N", title="County"),
    opacity=alt.condition(selopac, alt.value(1), alt.value(0.1))
).add_selection(selopac)

In [ ]:
chart_list = []
for county in list(biggest_county_jumps.tail(12).county):
    this_df = chronological_county_df[chronological_county_df.county == county]
    chart = alt.Chart(this_df, title=county).encode(
        x=alt.X("date:T", title=None, axis=None),
    )
    lines = chart.mark_line(color=lat.palette["accent"]).encode(
        y=alt.Y("new_confirmed_cases_rolling_average:Q", title=None,),
    )
    bars = chart.mark_bar(color=lat.palette["default"], opacity=0.33).encode(
        y=alt.Y("new_confirmed_cases:Q", title="New confirmed cases",),
    )
    chart_list.append((bars + lines).properties(height=200, width=250))
facet_wrap(chart_list, plots_per_row=4)

In [ ]:
import ipywidgets

In [ ]:
# for county in list(biggest_county_jumps.tail(12).county):
def countyplot(county='Los Angeles'):
    this_df = chronological_county_df[chronological_county_df.county == county]
    chart = alt.Chart(this_df, title=county).encode(
        x=alt.X("date:T", title=None, axis=None),
    )
    lines = chart.mark_line(color=lat.palette["accent"]).encode(
        y=alt.Y("new_confirmed_cases_rolling_average:Q", title=None,),
    )
    bars = chart.mark_bar(color=lat.palette["default"], opacity=0.33).encode(
        y=alt.Y("new_confirmed_cases:Q", title="New confirmed cases",),
    )
    #chart_list.append((bars + lines).properties(height=200, width=250))
    return (bars + lines).properties(height=200, width=250)
    
#facet_wrap(chart_list, plots_per_row=4)
ipywidgets.interact(countyplot,county=biggest_county_jumps.county.unique());